# Goal
- first steps with simpy
- feature out Simpy events, process, timeout and conditions

## prerequites
- basic Python knowledge
- Some OOP (Object Oriented Programming) knowledge

# What is Simpy

Simpy is a discrete simulation tool. 

TODO intro

## Install the simpy package

In [1]:
!pip install simpy

## Let us build a very basic simulation

We first need a Simpy environment. It is the frame of the simulation.

The main role of the environement is to keep track of processes and time passing

In [2]:
from simpy import Environment

In [3]:
# create an environment
env = Environment()
# rn the simulation for 10 steps
# now hold the current step of the simulation
print(f'Simulation starts at {env.now}')
sim_duration = 10
env.run(until=sim_duration)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Simulation stops at 10


It starts and stop after 10 steps.

## Add some people visiting the bank

Something happen in the simulation because an event has triggered. 
There are multiple types of events.

The documentation tells there are these kind of event 
```
events.Event 
| +— events.Timeout 
| +— events.Initialize 
| +— events.Process 
| +— events.Condition 
| | | +— events.AllOf 
| | | +— events.AnyOf 
```
TODO link to API reference

Let us start with process.

Processes are used to represent people ou objects interacting with the environnement. Their behavior is implemented as a process.

The visitor of the bank is a process. Let us assiume for the time being that the visitor enter the bank, have a look at this beatiful building and leave after a while. 

In [4]:
from simpy import Environment, Process

In [5]:
# This class defines the behavior of visitors
# it relies on the Process class
class Visitor(Process):  
    # This method constructs a visitor object
    def __init__(self, env, name):
        # first init the undelying process
        # the second argument is the action that must be taken
        super().__init__(env, self.visit())
        # give the visitor a name
        self.name = name

    # This method implements the visitor action
    # It must be a generator (more on thos in tuto-02)
    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now}") 
        # duration is the time the visitor wait inside the bank
        # For the sake of simplicity is it the length of the name
        duration = len(self.name)
        # We will see in tuto-02 what the yield thing does
        # timeout generate an event that last for the lengh of duration
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now}") 

In [6]:
# create an environment
env = Environment()
# create a visitor
Visitor(env, "Alice")
# run the simulation for 1à steps
print(f'Simulation starts at {env.now}')
sim_duration = 10
env.run(until=sim_duration)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Alice: Here I am at 0
Alice: I must leave at 5
Simulation stops at 10


The trace shows that Alice started a step 0 and leaved at step 5 after a timeout of 5  (tge name Alice has 5 letters)

Process objecrs like visitor or Timeout objects generate events. The environement keeps track of time and events. 

Events are triggered at some point in time. They start and after a while they may succeed or fail and end.

Event might be combined in a chain. 
- the process visit starts when visitor is created
- the environnement waits until it ends
- The process visit triggered a timeout event and waits until it ends


TODO event queue

## Internals of an event

Simpy let us add callbacks to events. 
The callback is a function that will be called when the event is completed.
This allows for running some extra processing at that point.

Let us add a callback to inspect the visitor at different point in time.

In [7]:
def snoop(event):
    print(f'Called back at {event.env.now}')
    print(f"event {event}")
    print(f"event triggered:{event.triggered} processed:{event.processed}")
    print(f'enf of callback')

class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now}") 
        snoop(self)
        duration = len(self.name)
        yield env.timeout(duration)
        snoop(self)
        print(f"{self.name}: I must leave at {self.env.now}") 
        
env = Environment()
visited = Visitor(env, "Alice")
# call the snoop function before the simulation starts
snoop(visited)
# set snoop as a callback to be called when the visit ends
visited.callbacks.append(snoop)
print(f'Simulation starts at {env.now}')
sim_duration = 10
env.run(until=sim_duration)
print(f'Simulation stops at {env.now}')

Called back at 0
event <Visitor(visit) object at 0x1122eb580>
event triggered:False processed:False
enf of callback
Simulation starts at 0
Alice: Here I am at 0
Called back at 0
event <Visitor(visit) object at 0x1122eb580>
event triggered:False processed:False
enf of callback
Called back at 5
event <Visitor(visit) object at 0x1122eb580>
event triggered:False processed:False
enf of callback
Alice: I must leave at 5
Called back at 5
event <Visitor(visit) object at 0x1122eb580>
event triggered:True processed:True
enf of callback
Simulation stops at 10


TODO description

## conditions and multiple conditions

Up to now the simulation ran for a predefined number of steps.

In the run expression below, the number stands for until the event env.timeout(duration) is done. 
```
env.run(until=sim_duration) 
```

However the parameter until expects any event or condition. The simulation might stop when a condition is met.

For instance, let us stop the simulation when Alice has visited.

In [8]:
class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now}") 
        duration = len(self.name)
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now}") 

In [9]:
env = Environment()
alice_visited = Visitor(env, "Alice")
print(f'Simulation starts at {env.now}')
# run the simulation until Alice's visit is done
env.run(until=alice_visited)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Alice: Here I am at 0
Alice: I must leave at 5
Simulation stops at 5


Please not that the simulation now ends at 5, right after the visit.

Events might be combined to form conditions in two ways
- AnyOf or | : event is done when one of the events is completed
- AllOf or & : event is done when all the events are completed
    

What if the simulation ends if either Alice's or Bob's visit is done 

In [10]:
env = Environment()
alice_visited = Visitor(env, "Alice")
bob_visited = Visitor(env, "Bob")
print(f'Simulation starts at {env.now}')
env.run(until= alice_visited | bob_visited)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Alice: Here I am at 0
Bob: Here I am at 0
Bob: I must leave at 3
Simulation stops at 3


The simulation ends at 3, when Bob's visit ends (the shortest) 

What if the simulation ends when both Alice's or Bob's visit are done 

In [11]:
env = Environment()
alice_visited = Visitor(env, "Alice")
bob_visited = Visitor(env, "Bob")
print(f'Simulation starts at {env.now}')
env.run(until= alice_visited & bob_visited)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Alice: Here I am at 0
Bob: Here I am at 0
Bob: I must leave at 3
Alice: I must leave at 5
Simulation stops at 5


The simulation ends at 5, when the longest visit ends.

## Condition in processes

Condition are also useful to model joins or alternatives in processes.

We will slighly change the model. Now the visit ends when the visitor has either taken a photo or is bored with the visit.

In [12]:
class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now}") 
        duration = len(self.name)
        # env.process is used to make a process out of the function
        bored = self.env.process(self.bored())
        got_picture = self.env.process(self.got_picture()) 
        # visit ends when either of those occurs                              #
        yield bored | got_picture
        print(f"{self.name}: I must leave at {self.env.now}") 
    
    def bored(self): 
        # get bored after 3 for Bob, and 5 for Alice
        duration = len(self.name)
        yield env.timeout(duration)
    
    def got_picture(self): 
        # taking the picture last 4
        yield env.timeout(4)


In [13]:
env = Environment()
alice_visited = Visitor(env, "Alice")
bob_visited = Visitor(env, "Bob")
print(f'Simulation starts at {env.now}')
# simulation stops when both visits end
env.run(until= alice_visited & bob_visited)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Alice: Here I am at 0
Bob: Here I am at 0
Bob: I must leave at 3
Alice: I must leave at 4
Simulation stops at 4


Bob leaved at 3. He got bored at 3 and would have had the photo at 4. bored is enough to complete the visit. 

Alice leaved at 4. She got the photo at 4 and would have been bored at 5. got_picture is enough to complete the visit.

## Home work

implement a visit when both conditions are met.

Which operation best reflect the time spent : 
    - sum of the duration of each sub process
    - max of the duration the sub processes

## References

TODO API 
TODO concepts